# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

# Оглавление
1.[Открытие данных](#start)<br>
2.[Предобработка данных](#preprocessing)<br>
    * [Обработка пропущенных значений](#null)<br>
    * [Замена типа данных](#data_change )<br>
    * [Обработка дубликатов](#duplicates)<br>
    * [Лемматизация](#lemma)<br>
    * [Категоризация данных](#category)<br>
3.[Ответьте на вопросы](#questions)<br>
    * [Есть ли зависимость между наличием детей и возвратом кредита в срок?](#q1)<br>
    * [Есть ли зависимость между семейным положением и возвратом кредита в срок?](#q2)<br>
    * [Есть ли зависимость между уровнем дохода и возвратом кредита в срок?](#q3)<br>
    * [Как разные цели кредита влияют на его возврат в срок?](#q4)<br>
4.[Общий вывод](#summary)<br>


## Шаг 1. Откройте файл с данными и изучите общую информацию
<a id="start"></a>

In [1]:
import pandas as pd
from pymystem3 import Mystem
m = Mystem()
from collections import Counter

In [2]:
data= pd.read_csv('/datasets/data.csv')
#изучаем таблицу, пока бегло
display(data.head(10))
print(data.info()) #изучаем таблицу/Замечаем пропуски в колонках 'days_employed' и 'total_income'


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
None


Наш датасет `data` состоит из 12 колонок и 21525 строк. Уже при первом взгляде видны ошибки. Рассмотрим  каждую колонку по отдельности.

In [3]:
data[(data['income_type']=='пенсионер')&(data['dob_years']!=0)]['dob_years'].median()

60.0

In [4]:
#смотрим уникальные значения в столбце "children"
#print('Количество детей\n',data['children'].value_counts()) 
#замечаем значения ""-1" и "20". Значения ""-1" вероятно опечатка от "1". 20 вероятно опечатка от "2",
#лишний "0".Заменим эти значения на верные.
data['children']=data['children'].replace(-1, 1)
data['children']=data['children'].replace(20, 2)

#Вычислим медианный возраст для каждой из нужных нам групп и округлим до целых
median_dob_years_sotrudnik = int(data[(data['income_type']=='сотрудник')&(data['dob_years']!=0)]['dob_years'].median())
median_dob_years_pensioner= int(data[(data['income_type']=='пенсионер')&(data['dob_years']!=0)]['dob_years'].median())
median_dob_years_companion= int(data[(data['income_type']=='компаньон')&(data['dob_years']!=0)]['dob_years'].median())
median_dob_years_gossluz = int(data[(data['income_type']=='госслужащий')&(data['dob_years']!=0)]['dob_years'].median())
#посмотрим на результат
print('сотрудник', median_dob_years_sotrudnik,'пенсионер',median_dob_years_pensioner,'компаньон',median_dob_years_companion,'госслужащий',median_dob_years_gossluz)


сотрудник 39 пенсионер 60 компаньон 39 госслужащий 40


In [5]:
#подставим значения этих переменных  в соответствующие ячейки.Напишем функцию и добавим ее методом apply 
def wrong_age(row):
    years = row['dob_years']
    income = row['income_type']

    if (years == 0) & (income=='сотрудник'):
        return median_dob_years_sotrudnik
    if (years == 0) & (income=='пенсионер'):
        return median_dob_years_pensioner
    if (years == 0) & (income=='компаньон'):
        return median_dob_years_companion
    if (years == 0) & (income=='госслужащий'):
        return median_dob_years_gossluz
    return years

data['dob_years']=data.apply(wrong_age, axis=1)
data['education'] = data['education'].str.lower()
data['family_status'] = data['family_status'].str.lower()
data['gender']=data['gender'].replace('XNA', 'F')
data['total_income']=data['total_income'].round()
display(data.head(10))


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253876.0,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.0,приобретение автомобиля
2,0,-5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145886.0,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267629.0,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.0,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255764.0,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240526.0,операции с жильем
7,0,-152.779569,50,среднее,1,женат / замужем,0,M,сотрудник,0,135824.0,образование
8,2,-6929.865299,35,высшее,0,гражданский брак,1,F,сотрудник,0,95857.0,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144426.0,покупка жилья для семьи


На данном этапе мы выполнили первичные преобразования со всеми колонками, кроме`days_employed`. Будем работать с ней в следующем блоке, т.к. она требует больше всего преобразований.

In [6]:
#смотрим минимальное значение в столбце "общий трудовой стаж в днях"
print('Минимальный трудовой стаж клиента',data['days_employed'].min())

Минимальный трудовой стаж клиента -18388.949900568383


In [7]:
#значения отрицательные. Так быть не может. Проверим их общее количество по отношению к длине столбца
print(data[(data['days_employed']<0)]['days_employed'].count()/len(data['days_employed']))

0.7389547038327526


In [8]:
#Их около 74%. Похоже ошибка в знаке минус, потому что по модулю отрицательного значению минмального значение
#при переводе в года получается 50 лет, что вполне возможно
#Заменим значения на противоположные по знаку
data['days_employed'] = data['days_employed'].apply(lambda i : i*(-1) if i < 0 else i)
#проверим количество строк с отрицательными значениями.
print(data[(data['days_employed']<0)]['days_employed'].count())


0


In [9]:
#округлим значения до целых
data['days_employed']=data['days_employed'].round()
#смотрим максимальное значение в столбце "общий трудовой стаж в днях"
print('Максимальный трудовой стаж клиента',data['days_employed'].max())

Максимальный трудовой стаж клиента 401755.0


In [10]:


#Выдача 401755.0, что в годах составляет 1100 лет.
#Надо посмотреть поближе на максимальные значения столбца, для этого отсортируем его по убыванию
data.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.479721,66914.727973,43.496167,0.817236,0.972544,0.080883,1.674223e+05
std,0.755528,139030.879631,12.231538,0.548138,1.420324,0.272661,1.029716e+05
min,0.000000,24.000000,19.000000,0.000000,0.000000,0.000000,2.066700e+04
25%,0.000000,927.000000,34.000000,1.000000,0.000000,0.000000,1.030530e+05
50%,0.000000,2194.000000,43.000000,1.000000,0.000000,0.000000,1.450180e+05
75%,1.000000,5538.000000,53.000000,1.000000,1.000000,0.000000,2.034350e+05
max,5.000000,401755.000000,75.000000,4.000000,4.000000,1.000000,2.265604e+06


In [11]:
#Поищем сколько таких "неразумных значений". У человека вряд ли может быть больше 70 лет стажа,
#даже в экстремальных случаях  70 х 365 = 25 550 дней.  
#Посчитаем строки со значеникм больше данного.
print(data[(data['days_employed']>25550)]['days_employed'].count())

3445


In [12]:
#получаем 3445 такимх значений
#Попробуем найти границу аномальной зоны
print(data[(data['days_employed']>15550)]['days_employed'].count())

3454


In [13]:

#получаем 3454 таких значений. Значений стало больше всего на 9 шт.Можно условно считать что граница на 25550
#смотрим на таблицу с аномальными значениями, чтобы сравнить значения по отношению к соседним колонкам
display(data[(data['days_employed']>25550)].sort_values(by = 'days_employed', ascending = False).head())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
6954,0,401755.0,56,среднее,1,вдовец / вдова,2,F,пенсионер,0,176278.0,ремонт жилью
10006,0,401716.0,69,высшее,0,не женат / не замужем,4,F,пенсионер,0,57390.0,получение образования
7664,1,401675.0,61,среднее,1,женат / замужем,0,F,пенсионер,0,126215.0,операции с жильем
2156,0,401674.0,60,среднее,1,женат / замужем,0,M,пенсионер,0,325396.0,автомобили
7794,0,401664.0,61,среднее,1,гражданский брак,1,F,пенсионер,0,48286.0,свадьба


In [14]:
#Была гипотеза  о часах, не переведенных в дни. Но не сработала,так как в выдаче
#есть клиенты возрастом 32 года и 328827.0 в столбце 'days_employed'
#что при гипотезе данной ошибки дает  328827/365/24 = 37,5 лет (больше возраста клиента)
#Далее встречаются аналогичные значения.
#Установить причину аномалии проблематично без контакта с составителем датасета, поэтому скорректируем эти значения.
#как медиану по категории 'income_type' (как вариант)
#создадим вспомогательные массивы с аномальными и нормальными значениями 'days-employed'.
#добавим copy() чтобы не выпадало предупреждение SettingWithCopyWarning при попытке перезаписать эти массивы
dataset_anomal = data[(data['days_employed']>=25550)].copy()
dataset_normal =data[(data['days_employed']<25550)].copy()
#посмотрим на значения 'income_type' в аномлаьном массиве
print(dataset_anomal['income_type'].value_counts())

пенсионер      3443
безработный       2
Name: income_type, dtype: int64


In [15]:
#посмотрим на значения 'income_type' в нормальном массиве
print(dataset_normal['income_type'].value_counts())

сотрудник          10014
компаньон           4577
госслужащий         1312
предприниматель        1
в декрете              1
студент                1
Name: income_type, dtype: int64


In [16]:
#Как видим, пенсионеры и безработные есть только в аномальном массиве (не считая пропущенных значений, они нас пока не интересуют)
#Значит ,по столбцу 'income_type' мы не скорректируемся.
##Посмотрим на данные - есть ли колонки где людям много лет и они не пенсионеры.
display(dataset_normal[(dataset_normal['income_type']!= 'пенсионер')&((dataset_normal['dob_years']> 50))].sort_values(by = 'days_employed', ascending = False).sample(10))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
6986,0,6089.0,56,высшее,0,женат / замужем,0,F,сотрудник,0,220458.0,получение высшего образования
19660,1,441.0,60,неоконченное высшее,2,в разводе,3,M,сотрудник,0,128055.0,недвижимость
4158,0,1544.0,51,среднее,1,не женат / не замужем,4,F,сотрудник,0,110513.0,на покупку автомобиля
12965,0,4216.0,59,высшее,0,в разводе,3,F,компаньон,0,653203.0,строительство жилой недвижимости
19694,0,8379.0,54,среднее,1,гражданский брак,1,M,компаньон,0,114310.0,строительство недвижимости
9099,0,1651.0,53,среднее,1,гражданский брак,1,M,сотрудник,0,123148.0,жилье
14261,0,2888.0,55,среднее,1,женат / замужем,0,M,сотрудник,0,114170.0,жилье
9579,0,3684.0,51,среднее,1,вдовец / вдова,2,F,сотрудник,0,62454.0,операции со своей недвижимостью
13554,0,1061.0,51,среднее,1,женат / замужем,0,F,компаньон,0,139936.0,на покупку автомобиля
6398,0,2064.0,51,высшее,0,не женат / не замужем,4,F,компаньон,0,177265.0,покупка своего жилья


In [17]:
#Такие клиенты есть. 
#Тогда надо добавить колонку с декадой возраста. В зависимости от числа лет, она вернет декаду.
#19 лет - 2я декада, 35 лет -4я, 75 лет - 8я и т.д.
#напишем функцию, которая это делает.
def decade_age(age):
    if 10<age<20:
        return 2
    if 20<=age<30:
        return 3
    if 30<=age<40:
        return 4
    if 40<=age<50:
        return 5
    if 50<=age<60:
        return 6
    if 60<=age<70:
        return 7
    if 70<=age<80:
        return 8
#проверим ее    
print(decade_age(12)) #Вывод 2
print(decade_age(50)) #Вывод 6
print(decade_age(70)) #Вывод 8

2
6
8


In [18]:
#Добавим в нормальный и аномальный массивы данные колонки
dataset_normal['decade_age'] = dataset_normal['dob_years'].apply(decade_age)
dataset_anomal['decade_age'] = dataset_anomal['dob_years'].apply(decade_age)
data['decade_age'] = data['dob_years'].apply(decade_age)

In [19]:

#проверим, добавились ли колонки
display(dataset_normal.head(10))
display(dataset_anomal.head(10))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,decade_age
0,1,8438.0,42,высшее,0,женат / замужем,0,F,сотрудник,0,253876.0,покупка жилья,5
1,1,4025.0,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.0,приобретение автомобиля,4
2,0,5623.0,33,среднее,1,женат / замужем,0,M,сотрудник,0,145886.0,покупка жилья,4
3,3,4125.0,32,среднее,1,женат / замужем,0,M,сотрудник,0,267629.0,дополнительное образование,4
5,0,926.0,27,высшее,0,гражданский брак,1,M,компаньон,0,255764.0,покупка жилья,3
6,0,2879.0,43,высшее,0,женат / замужем,0,F,компаньон,0,240526.0,операции с жильем,5
7,0,153.0,50,среднее,1,женат / замужем,0,M,сотрудник,0,135824.0,образование,6
8,2,6930.0,35,высшее,0,гражданский брак,1,F,сотрудник,0,95857.0,на проведение свадьбы,4
9,0,2189.0,41,среднее,1,женат / замужем,0,M,сотрудник,0,144426.0,покупка жилья для семьи,5
10,2,4171.0,36,высшее,0,женат / замужем,0,M,компаньон,0,113943.0,покупка недвижимости,4


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,decade_age
4,0,340266.0,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.0,сыграть свадьбу,6
18,0,400281.0,53,среднее,1,вдовец / вдова,2,F,пенсионер,0,56824.0,на покупку подержанного автомобиля,6
24,1,338552.0,57,среднее,1,не женат / не замужем,4,F,пенсионер,0,290547.0,операции с коммерческой недвижимостью,6
25,0,363548.0,67,среднее,1,женат / замужем,0,M,пенсионер,0,55113.0,покупка недвижимости,7
30,1,335582.0,62,среднее,1,женат / замужем,0,F,пенсионер,0,171456.0,операции с коммерческой недвижимостью,7
35,0,394021.0,68,среднее,1,гражданский брак,1,M,пенсионер,0,77806.0,на проведение свадьбы,7
50,0,353731.0,63,среднее,1,женат / замужем,0,F,пенсионер,0,92343.0,автомобили,7
56,0,370145.0,64,среднее,1,вдовец / вдова,2,F,пенсионер,0,149141.0,образование,7
71,0,338114.0,62,среднее,1,женат / замужем,0,F,пенсионер,0,43930.0,автомобили,7
78,0,359723.0,61,высшее,0,женат / замужем,0,M,пенсионер,0,175128.0,сделка с автомобилем,7


In [20]:
medians_days_employed_anomal = dataset_anomal.groupby('decade_age')['days_employed'].median().round()
medians_days_employed_normal = dataset_normal.groupby('decade_age')['days_employed'].median().round()

#посмотрим на эти медианы
print(medians_days_employed_anomal)
print(medians_days_employed_normal)

decade_age
3    364348.0
4    364039.0
5    367508.0
6    364343.0
7    365501.0
8    366504.0
Name: days_employed, dtype: float64
decade_age
2     724.0
3    1000.0
4    1586.0
5    2022.0
6    2262.0
7    2669.0
8    2680.0
Name: days_employed, dtype: float64


In [21]:

#Получим корректирующие коэффициенты на которые поделим аномальные значения
k3=medians_days_employed_anomal[3]/medians_days_employed_normal[3]
k4=medians_days_employed_anomal[4]/medians_days_employed_normal[4]
k5=medians_days_employed_anomal[5]/medians_days_employed_normal[5]
k6=medians_days_employed_anomal[6]/medians_days_employed_normal[6]
k7=medians_days_employed_anomal[7]/medians_days_employed_normal[7]
k8=medians_days_employed_anomal[8]/medians_days_employed_normal[8]

print('Коэффициенты',k3,k4,k5,k6,k7,k8)

#скорректируем аномальные значения в первоначальном датасете
data.loc[(data['days_employed']> 25500 )&(data['decade_age']==3),'days_employed'] = data.loc[(data['days_employed']> 25500 )&(data['decade_age']==3),'days_employed'].map(lambda x:x/k3).round()
data.loc[(data['days_employed']> 25500 )&(data['decade_age']==4),'days_employed']=data.loc[(data['days_employed']> 25500 )&(data['decade_age']==4),'days_employed'].map(lambda x:x/k4).round()
data.loc[(data['days_employed']> 25500 )&(data['decade_age']==5),'days_employed']=data.loc[(data['days_employed']> 25500 )&(data['decade_age']==5),'days_employed'].map(lambda x:x/k5).round()
data.loc[(data['days_employed']> 25500 )&(data['decade_age']==6),'days_employed']=data.loc[(data['days_employed']> 25500 )&(data['decade_age']==6),'days_employed'].map(lambda x:x/k6).round()
data.loc[(data['days_employed']> 25500 )&(data['decade_age']==7),'days_employed']=data.loc[(data['days_employed']> 25500 )&(data['decade_age']==7),'days_employed'].map(lambda x:x/k7).round()
data.loc[(data['days_employed']> 25500 )&(data['decade_age']==8),'days_employed']=data.loc[(data['days_employed']> 25500 )&(data['decade_age']==8),'days_employed'].map(lambda x:x/k8).round()

#Проверим резальтат
display(data.sort_values(by = 'days_employed', ascending = False).head(10))
#Максимальное значение 18389.0, замена выполнена
#К выводам - кластер пенсионеров и безработных ошибочный
#К выводам - стаж вообще странный, может это стаж на последнем месте работы? Он подозрительно маленький в днях при переводе в годы
#все равно остаются странные стажи - например стаж в 50 лет при возрасте в 61 год. Хотя возможно возраста там не было и мы его подставили.


Коэффициенты 364.348 229.5327868852459 181.75469831849654 161.0711759504863 136.94304983139753 136.75522388059701


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,decade_age
16335,1,18389.0,61,среднее,1,женат / замужем,0,F,сотрудник,0,186179.0,операции с недвижимостью,7
4299,0,17616.0,61,среднее,1,женат / замужем,0,F,компаньон,0,122561.0,покупка жилья,7
7329,0,16593.0,60,высшее,0,женат / замужем,0,F,сотрудник,0,124698.0,заняться высшим образованием,7
17838,0,16265.0,59,среднее,1,женат / замужем,0,F,сотрудник,0,51239.0,на покупку автомобиля,6
16825,0,16120.0,64,среднее,1,женат / замужем,0,F,сотрудник,0,91528.0,покупка жилой недвижимости,7
3974,0,15836.0,64,среднее,1,гражданский брак,1,F,компаньон,0,96859.0,сыграть свадьбу,7
1539,0,15786.0,59,высшее,0,не женат / не замужем,4,F,сотрудник,0,119564.0,операции с коммерческой недвижимостью,6
4321,0,15773.0,61,среднее,1,гражданский брак,1,F,сотрудник,0,205869.0,свадьба,7
7731,0,15618.0,64,среднее,1,женат / замужем,0,F,компаньон,0,296525.0,высшее образование,7
15675,0,15410.0,65,высшее,0,женат / замужем,0,F,сотрудник,0,188800.0,покупка жилой недвижимости,7


**Вывод**

В блоке выше выполнили первоначальную обработку столбца `days_employed`. Мы привели его в максимально правдоподобный вид, с учетом ограниченной информации, но в реальных условиях лучше уточнять по нему данные у разработчиков датасета. В нем было слишком много ошибок, и возможно мы применяли неверные гипотезы для их устранения

## Шаг 2. Предобработка данных
<a id="preprocessing"></a>

### Обработка пропусков
<a id="null"></a>

In [22]:
#Посмотрим на количество пропусков в колонках.
print(data.isnull().mean()) 

children            0.000000
days_employed       0.100999
dob_years           0.000000
education           0.000000
education_id        0.000000
family_status       0.000000
family_status_id    0.000000
gender              0.000000
income_type         0.000000
debt                0.000000
total_income        0.100999
purpose             0.000000
decade_age          0.000000
dtype: float64


In [23]:

#Видим что число пропусков в колонках 'days_employed' и 'total_income' одинаковое, около 10%,
#значит в этих строках нет обоих параметров
#смотрим на строки с пропусками, пытаемся понять логику пропусков. 
display(data[(data['total_income'].isnull()==True)]) 

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,decade_age
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу,7
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование,5
29,0,NaN,63,среднее,1,не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости,7
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем,6
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,среднее,1,женат / замужем,0,M,компаньон,0,NaN,сделка с автомобилем,5
21495,1,NaN,50,среднее,1,гражданский брак,1,F,сотрудник,0,NaN,свадьба,6
21497,0,NaN,48,высшее,0,женат / замужем,0,F,компаньон,0,NaN,строительство недвижимости,5
21502,1,NaN,42,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,строительство жилой недвижимости,5


In [24]:
#Связи с сосденими столбцами не видно.
#Заменим пропуски на медианные значения по категориям. Начем с 'total_income', так как это более простая задача
#посмотрим уникальные значения 'income_type'для строк с пропусками
print(data[(data['total_income'].isnull()==True)]['income_type'].value_counts())

сотрудник          1105
компаньон           508
пенсионер           413
госслужащий         147
предприниматель       1
Name: income_type, dtype: int64


In [25]:
#значит нам нужны медианные ежемесячные доходы для этих 5 категорий
#Вычислим медианный доход для каждой категории
medians_total_income = data.groupby('income_type')['total_income'].median().round()
print('Медианы----------',medians_total_income)
#Возьмем из этого массива нужные нам категории
median_total_income_sotrudnik = medians_total_income[6]
median_total_income_companion= medians_total_income[3]
median_total_income_pensioner= medians_total_income[4]
median_total_income_gossluz = medians_total_income[2]
median_total_income_predprinimatel= medians_total_income[5]
#проверим что взяли правлиьные
print('сотрудник', median_total_income_sotrudnik,'компаньон',median_total_income_companion,
      'пенсионер',median_total_income_pensioner,'госслужащий',median_total_income_gossluz,'предприниматель',median_total_income_predprinimatel)

Медианы---------- income_type
безработный        131340.0
в декрете           53829.0
госслужащий        150448.0
компаньон          172358.0
пенсионер          118514.0
предприниматель    499163.0
сотрудник          142594.0
студент             98202.0
Name: total_income, dtype: float64
сотрудник 142594.0 компаньон 172358.0 пенсионер 118514.0 госслужащий 150448.0 предприниматель 499163.0


In [26]:
#заменим пропуски 'total_income' медианами
data.loc[(data['total_income'].isnull())& (data['income_type'] == 'сотрудник'),'total_income']=median_total_income_sotrudnik
data.loc[(data['total_income'].isnull())& (data['income_type'] == 'компаньон'),'total_income']=median_total_income_companion
data.loc[(data['total_income'].isnull())& (data['income_type'] == 'пенсионер'),'total_income']=median_total_income_pensioner
data.loc[(data['total_income'].isnull())& (data['income_type'] == 'госслужащий'),'total_income']=median_total_income_gossluz
data.loc[(data['total_income'].isnull())& (data['income_type'] == 'предприниматель'),'total_income']=median_total_income_predprinimatel
#проверим результат
print(data.isnull().mean())

children            0.000000
days_employed       0.100999
dob_years           0.000000
education           0.000000
education_id        0.000000
family_status       0.000000
family_status_id    0.000000
gender              0.000000
income_type         0.000000
debt                0.000000
total_income        0.000000
purpose             0.000000
decade_age          0.000000
dtype: float64


In [27]:

#займемся столбцом days_employed
#скопируем датасет, чтобы не добавлять в оригинальный лишние столбцы
data_1 =data.copy()
#добавим в data_1 декаду возраста,фукнция описана в предыдущих блоках
data_1['decade_age'] = data_1['dob_years'].apply(decade_age)
#посчитаем медиану
medians_days_employed_all = data_1.groupby('decade_age')['days_employed'].median().round()
#выведеим медиану
print(medians_days_employed_all)
#заполним пропуски в days_employed медианами по возрасту
data.loc[(data['days_employed'].isnull())& (data['decade_age'] == 2),'days_employed']=medians_days_employed_all[2]
data.loc[(data['days_employed'].isnull())& (data['decade_age'] == 3),'days_employed']=medians_days_employed_all[3]
data.loc[(data['days_employed'].isnull())& (data['decade_age'] == 4),'days_employed']=medians_days_employed_all[4]
data.loc[(data['days_employed'].isnull())& (data['decade_age'] == 5),'days_employed']=medians_days_employed_all[5]
data.loc[(data['days_employed'].isnull())& (data['decade_age'] == 6),'days_employed']=medians_days_employed_all[6]
data.loc[(data['days_employed'].isnull())& (data['decade_age'] == 7),'days_employed']=medians_days_employed_all[7]
data.loc[(data['days_employed'].isnull())& (data['decade_age'] == 8),'days_employed']=medians_days_employed_all[8]
#проверим результат
print(data.isnull().sum())
#пропусков не осталось


decade_age
2     724.0
3    1000.0
4    1586.0
5    2022.0
6    2262.0
7    2669.0
8    2680.0
Name: days_employed, dtype: float64
children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
decade_age          0
dtype: int64


**Вывод**

Обнаружили пропущенный значения типа NaN в столбцах 'days_employed' и 'total_income', причем с одинаковым количеством. Возможная причина появления - отсутствие документального подверждения дохода и стажа.
Заполнили пропуски в 'days_employed' на медианное значение по декаде возраста, в 'total_income' на медианное значение по 'income_type'.

### Замена типа данных
<a id="data_change"></a>

In [28]:
#смотрим типы данных
print(data.info())
#очевидно нужно заменить тип данных в 'days_employed' и  'total_income'
data['days_employed'] = pd.to_numeric(data['days_employed'], errors='coerce') #
data['total_income'] = pd.to_numeric(data['total_income'], errors='coerce') #
#проверим результат
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 13 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
decade_age          21525 non-null int64
dtypes: float64(2), int64(6), object(5)
memory usage: 2.1+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 13 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
educat

**Вывод**

Обнаружены 2 проблемные колонки 'days_employed' и 'total_income' с типом данных object. Эти данные типа object, а нужны числовые. Заменили тип данных на float64. Применен метод pd.to_numeric с параметром errors='coerce'

### Обработка дубликатов
<a id="duplicates"></a>

In [29]:
#посмотрим количество строк дубликатов
print(data.duplicated().sum()) 
print(data.shape)
#сбросим дубликаты
data = data.drop_duplicates().reset_index(drop=True)
#проверим результат
print(data.duplicated().sum()) 
print(data.shape)

71
(21525, 13)
0
(21454, 13)


**Вывод**

Сначала удалили полные дубликаты.
Возможны неявные дубликаты, т.е. по колонкам без цифр, их несколько тысяч. Но это вопрос спорный, возможно теоретически совпадение по 7 пунктам,нужно запрашивать составителей датасета.
Возможные причины дубликатов: 
1)Разные люди работали над одним датасетом
2)Человек заполнял анкету несколько раз, при попытке взять кредит спустя некоторое время.
3)Ошибка в заполннении.

### Лемматизация
<a id="lemma"></a>


In [30]:

#посмотрим уникальные значение колонки 'purpose'
print(data['purpose'].unique())

#посмотрим количество уникальных значений
print()
print("Количество уникальных значений:",len(data['purpose'].unique()))

['покупка жилья' 'приобретение автомобиля' 'дополнительное образование'
 'сыграть свадьбу' 'операции с жильем' 'образование'
 'на проведение свадьбы' 'покупка жилья для семьи' 'покупка недвижимости'
 'покупка коммерческой недвижимости' 'покупка жилой недвижимости'
 'строительство собственной недвижимости' 'недвижимость'
 'строительство недвижимости' 'на покупку подержанного автомобиля'
 'на покупку своего автомобиля' 'операции с коммерческой недвижимостью'
 'строительство жилой недвижимости' 'жилье'
 'операции со своей недвижимостью' 'автомобили' 'заняться образованием'
 'сделка с подержанным автомобилем' 'получение образования' 'автомобиль'
 'свадьба' 'получение дополнительного образования' 'покупка своего жилья'
 'операции с недвижимостью' 'получение высшего образования'
 'свой автомобиль' 'сделка с автомобилем' 'профильное образование'
 'высшее образование' 'покупка жилья для сдачи' 'на покупку автомобиля'
 'ремонт жилью' 'заняться высшим образованием']

Количество уникальных значен

In [44]:
%%time
unique_purpose =data['purpose'].unique()
#объявим список куда будем заносить леммы
lemm_spisok=[]

for element in unique_purpose: 
      lemm_spisok.extend(m.lemmatize(element))

#соберем все леммы в одинк список в конйнер Counter, и посмотрим
print(Counter(lemm_spisok))

Counter({' ': 59, '\n': 38, 'покупка': 10, 'недвижимость': 10, 'автомобиль': 9, 'образование': 9, 'жилье': 7, 'с': 5, 'операция': 4, 'на': 4, 'свой': 4, 'свадьба': 3, 'строительство': 3, 'получение': 3, 'высокий': 3, 'дополнительный': 2, 'для': 2, 'коммерческий': 2, 'жилой': 2, 'заниматься': 2, 'сделка': 2, 'приобретение': 1, 'сыграть': 1, 'проведение': 1, 'семья': 1, 'собственный': 1, 'подержать': 1, 'со': 1, 'подержанный': 1, 'профильный': 1, 'сдача': 1, 'ремонт': 1})
CPU times: user 4.47 ms, sys: 4.28 ms, total: 8.75 ms
Wall time: 12.8 ms


In [45]:
%%time
#исходя из результата, выделим основные категории кредита
options = ["сдача","коммерческий","недвижимость","автомобиль","образование","жилье","свадьба"]

#Напишем функцию'purpose_cat', чтобы добавить столбец с категорией кредита
def purpose_сat(purpose):
    fraza=m.lemmatize(purpose)
    for word in options:
        if word in fraza:
            fraza=word
    return fraza

CPU times: user 5 µs, sys: 1 µs, total: 6 µs
Wall time: 9.54 µs


In [46]:
%%time

#создадим новый столбец, применив эту функцию к столбцу purpose
data['purpose_cat']=data['purpose'].apply(purpose_сat)
#выведем на экран
display(data.sample(5))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,decade_age,purpose_cat
13849,1,2955.0,44,высшее,0,женат / замужем,0,F,компаньон,0,322801.0,покупка недвижимости,5,недвижимость
14836,1,5430.0,37,среднее,1,женат / замужем,0,M,компаньон,0,134471.0,сделка с автомобилем,4,автомобиль
17920,0,1000.0,21,высшее,0,не женат / не замужем,4,F,сотрудник,1,142594.0,строительство собственной недвижимости,3,недвижимость
5424,0,1194.0,56,среднее,1,гражданский брак,1,F,компаньон,0,139403.0,строительство недвижимости,6,недвижимость
5899,0,2262.0,50,среднее,1,в разводе,3,F,сотрудник,0,142594.0,на покупку своего автомобиля,6,автомобиль


CPU times: user 1.28 s, sys: 348 ms, total: 1.63 s
Wall time: 3.88 s


In [47]:
print(data['purpose_cat'].value_counts())

недвижимость    5040
автомобиль      4306
образование     4013
жилье           3809
свадьба         2324
коммерческий    1311
сдача            651
Name: purpose_cat, dtype: int64


In [48]:
#свернем категории следущим образом: "жилье" войдет в "недвижимость", "сдача" войдет в "коммерческий"  
data.loc[(data['purpose_cat'] == 'жилье'),'purpose_cat']='недвижимость'
data.loc[(data['purpose_cat'] == 'сдача'),'purpose_cat']='коммерческий'
#проверим результат
print(data['purpose_cat'].value_counts())

недвижимость    8849
автомобиль      4306
образование     4013
свадьба         2324
коммерческий    1962
Name: purpose_cat, dtype: int64


**Вывод**

Все категории можно свести к 5 основным, которые мы используем при ответе на вопрос о влиянии цели кредита и задолженности

### Категоризация данных
<a id="category"></a>

In [49]:
#создадим и выведем таблицы, которые нам понадобятся для ответов на вопросы

data_children=data[['children','debt']]
display(data_children.sample(5))

data_family_status=data[['family_status','family_status_id','debt']]
display(data_family_status.sample(5))
#ниже сделаем копию, так как будем добавлять столбец в эту таблицу, чтобы избежать ошибки
data_total_income=data[['total_income','debt']].copy()
display(data_total_income.sample(5))
data_purpose=data[['purpose_cat','debt']]
display(data_purpose.sample(5))
       

,children,debt
15889,1,0
16375,0,0
13394,1,0
8264,0,0
3727,0,0


,family_status,family_status_id,debt
495,женат / замужем,0,0
16635,в разводе,3,0
19329,гражданский брак,1,0
5221,женат / замужем,0,0
11596,гражданский брак,1,0


,total_income,debt
13415,271372.0,1
1334,87830.0,0
1284,195485.0,0
11896,142594.0,0
9682,151075.0,0


,purpose_cat,debt
12376,недвижимость,0
10202,недвижимость,0
15375,автомобиль,0
18527,образование,0
14709,недвижимость,0


**Вывод**

Выбрали словари соответственно с поставленными вопросами, в которых будут только нужные нам данные<br>
1)data_children -  количество детей и наличие задолженности<br>
2)data_family_status - семейный статус, семейный статус id и наличие задолженности<br>
3)data_total_income - ежемесячный доход и наличие задолженности<br>
4)data_children - категория цели кредита и наличие задолженности<br>
Удалять явные дубликаты мы не можем, так как у нас не задан явный id каждого клиента.<br>

## Шаг 3. Ответьте на вопросы
<a id="questions"></a>

- Есть ли зависимость между наличием детей и возвратом кредита в срок?
<a id="q1"></a>

In [50]:
#Объявим переменные, считающие количество клиентов с детьми, без детей, и общее число дожников
s_detmi= data_children[(data_children['children']>0)]['children'].count()
bez_detej= data_children[(data_children['children']==0)]['children'].count()
dolznik= data_children[(data_children['debt']==1)]['debt'].count()


print('Клиенты с детьми',s_detmi)
print('Клиенты без детей',bez_detej)

print('Процент клиентов с детьми и задолженностью в прошлом:{:.2%}'.format(data_children[(data_children['children']>0)&(data_children['debt']==1)]['children'].count()/s_detmi))
print( 'Процент клиентов без детей и задолженностью в прошлом:{:.2%}'.format(data_children[(data_children['children']==0)&(data_children['debt']==1)]['children'].count()/bez_detej))
#возможен поиск закономерностей при разном количестве детей, но задача не стоит
print('Процент клиентов с задолженностью в прошлом в общем:{:.2%}'.format(dolznik/len(data_children['debt'])))

Клиенты с детьми 7363
Клиенты без детей 14091
Процент клиентов с детьми и задолженностью в прошлом:9.21%
Процент клиентов без детей и задолженностью в прошлом:7.54%
Процент клиентов с задолженностью в прошлом в общем:8.12%


**Вывод**

Согласно полученным данным, между фактом наличия детей и возвратом кредита в срок есть небольшая зависимость. Клиенты без детей в среднем имеют меньше задолженность, чем клиенты с детьми - 7,54 % и 9,21%. Это говорит о том что отсутствие детей - благоприятный фактор для выдачи кредита клиенту, но не существунный.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?
<a id="q2"></a>

In [51]:
#Объявим переменные для разных семеный статусов. Выведем их
fmst0= data_family_status[(data_family_status['family_status_id']==0)]['family_status_id'].count()
fmst1= data_family_status[(data_family_status['family_status_id']==1)]['family_status_id'].count()
fmst2= data_family_status[(data_family_status['family_status_id']==2)]['family_status_id'].count()
fmst3= data_family_status[(data_family_status['family_status_id']==3)]['family_status_id'].count()
fmst4= data_family_status[(data_family_status['family_status_id']==4)]['family_status_id'].count()

print('Клиенты со статусом "женат / замужем"',fmst0)
print('Клиенты со статусом"гражданский брак"',fmst1)
print('Клиенты со статусом "вдовец"',fmst2)
print('Клиенты со статусом "в разводе"',fmst3)
print('Клиенты со статусом"не женат / не замужем"',fmst4)

Клиенты со статусом "женат / замужем" 12339
Клиенты со статусом"гражданский брак" 4151
Клиенты со статусом "вдовец" 959
Клиенты со статусом "в разводе" 1195
Клиенты со статусом"не женат / не замужем" 2810


In [52]:
#Выведем результат
print('Процент клиентов со статусом "женат / замужем" и задолженностью:{:.2%}'.format(data_family_status[(data_family_status['family_status_id']==0)&(data_children['debt']==1)]['family_status_id'].count()/fmst0))
print('Процент клиентов со статусом "гражданский брак" и задолженностью:{:.2%}'.format(data_family_status[(data_family_status['family_status_id']==1)&(data_children['debt']==1)]['family_status_id'].count()/fmst1))
print('Процент клиентов со статусом "вдовец" и задолженностью:{:.2%}'.format(data_family_status[(data_family_status['family_status_id']==2)&(data_children['debt']==1)]['family_status_id'].count()/fmst2))
print('Процент клиентов со статусом "в разводе" и задолженностью:{:.2%}'.format(data_family_status[(data_family_status['family_status_id']==3)&(data_children['debt']==1)]['family_status_id'].count()/fmst3))
print('Процент клиентов со статусом "не женат / не замужем" и задолженностью:{:.2%}'.format(data_family_status[(data_family_status['family_status_id']==4)&(data_children['debt']==1)]['family_status_id'].count()/fmst4))

print('Процент клиентов с задолженностью в прошлом в общем:{:.2%}'.format(dolznik/len(data_children['debt'])))


Процент клиентов со статусом "женат / замужем" и задолженностью:7.55%
Процент клиентов со статусом "гражданский брак" и задолженностью:9.35%
Процент клиентов со статусом "вдовец" и задолженностью:6.57%
Процент клиентов со статусом "в разводе" и задолженностью:7.11%
Процент клиентов со статусом "не женат / не замужем" и задолженностью:9.75%
Процент клиентов с задолженностью в прошлом в общем:8.12%


**Вывод**

Согласно полученным данным, между семейным положением и наличием задолженности в прошлом есть небольшая зависимость. Самый низкий процент задолженности у людей со статусом "вдовец" -6,57 %. Самый высокий процент задолженности у людей с семейным положением "не женат / не замужем" = 9,75%. Это говорит о том что семейное положение может быть использовано как фактор при рассмотрении решения о выдаче кредита.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
<a id="q3"></a>

In [53]:
#сортирум по уровню дохода
data_total_income.sort_values(by = 'total_income',ascending = False) 
#вычислим медианный доход, чтобы от него отталкиваться деля на группы
# В дальнейшем используем его в функции деления клиентов на категории по доходу
median_dohod=data_total_income['total_income'].median()
print('Медианный ежемесячный доход, рубли:{:.0f}'.format(median_dohod))
#Медианный ежемесячный доход, рубли:142 594
#поделим на 4 категории. До половины медианы - низки, от половины до целой - ниже среднего, от целой до полуторной
#-выше среднейго, выше полуторной - высокий
def kat_dohod(dohod):
    if dohod <= (71297):
        return 'низкий'
    if (71297) < dohod <= (142594):
        return 'ниже среднего'
    if (142594) < dohod < (213891):
        return 'выше среднего'    
    return 'высокий' 
#добавим столбец с уровнем дохода по категории
data_total_income['uroven_dohoda'] = data_total_income['total_income'].apply(kat_dohod) #добавим колонку с категорией ежемесячного дохода
#проверим результат
display(data_total_income.sample(20))


Медианный ежемесячный доход, рубли:142594


,total_income,debt,uroven_dohoda
12633,127584.0,0,ниже среднего
16482,65875.0,0,низкий
2202,145005.0,0,выше среднего
3077,211519.0,0,выше среднего
20442,238958.0,0,высокий
12273,128462.0,0,ниже среднего
6865,148930.0,1,выше среднего
21086,186010.0,0,выше среднего
19007,144049.0,0,выше среднего
14512,90820.0,0,ниже среднего


In [54]:

print('Число клиентов c уровнем дохода "низкий":{:.0f}'.format(data_total_income[data_total_income['uroven_dohoda']=='низкий']['debt'].count()))
print('Число клиентов с уровнем дохода "ниже среднего":{:.0f}'.format(data_total_income[data_total_income['uroven_dohoda']=='ниже среднего']['debt'].count()))
print('Число клиентов с уровнем дохода"выше среднего":{:.0f}'.format(data_total_income[data_total_income['uroven_dohoda']=='выше среднего']['debt'].count()))
print('Число клиентов с уровнем дохода "высокий":{:.0f}'.format(data_total_income[data_total_income['uroven_dohoda']=='высокий']['debt'].count()))
print()
print('Число клиентов с задолженностью c уровнем дохода "низкий":{:.0f}'.format(data_total_income[data_total_income['uroven_dohoda']=='низкий']['debt'].sum()))
print('Число клиентов с задолженностью с уровнем дохода "ниже среднего":{:.0f}'.format(data_total_income[data_total_income['uroven_dohoda']=='ниже среднего']['debt'].sum()))
print('Число клиентов с задолженностью с уровнем дохода"выше среднего":{:.0f}'.format(data_total_income[data_total_income['uroven_dohoda']=='выше среднего']['debt'].sum()))
print('Число клиентов с задолженностью с уровнем дохода "высокий":{:.0f}'.format(data_total_income[data_total_income['uroven_dohoda']=='высокий']['debt'].sum()))
print()

print('Процент клиентов с задолженностью c уровнем дохода "низкий":{:.2%}'.format(data_total_income[data_total_income['uroven_dohoda']=='низкий']['debt'].sum()/data_total_income[data_total_income['uroven_dohoda']=='низкий']['debt'].count()))
print('Процент клиентов с задолженностью с уровнем дохода "ниже среднего":{:.2%}'.format(data_total_income[data_total_income['uroven_dohoda']=='ниже среднего']['debt'].sum()/data_total_income[data_total_income['uroven_dohoda']=='ниже среднего']['debt'].count()))
print('Процент клиентов с задолженностью с уровнем дохода"выше среднего":{:.2%}'.format(data_total_income[data_total_income['uroven_dohoda']=='выше среднего']['debt'].sum()/data_total_income[data_total_income['uroven_dohoda']=='выше среднего']['debt'].count()))
print('Процент клиентов с задолженностью с уровнем дохода "высокий":{:.2%}'.format(data_total_income[data_total_income['uroven_dohoda']=='высокий']['debt'].sum()/data_total_income[data_total_income['uroven_dohoda']=='высокий']['debt'].count()))
print()
print('Процент клиентов с задолженностью в прошлом в общем:{:.2%}'.format(dolznik/len(data_children['debt'])))



Число клиентов c уровнем дохода "низкий":1579
Число клиентов с уровнем дохода "ниже среднего":9264
Число клиентов с уровнем дохода"выше среднего":6291
Число клиентов с уровнем дохода "высокий":4320

Число клиентов с задолженностью c уровнем дохода "низкий":109
Число клиентов с задолженностью с уровнем дохода "ниже среднего":801
Число клиентов с задолженностью с уровнем дохода"выше среднего":527
Число клиентов с задолженностью с уровнем дохода "высокий":304

Процент клиентов с задолженностью c уровнем дохода "низкий":6.90%
Процент клиентов с задолженностью с уровнем дохода "ниже среднего":8.65%
Процент клиентов с задолженностью с уровнем дохода"выше среднего":8.38%
Процент клиентов с задолженностью с уровнем дохода "высокий":7.04%

Процент клиентов с задолженностью в прошлом в общем:8.12%


**Вывод**

Согласно полученным данным, между уровнем дохода и наличием задолженности в прошлом есть небольшая зависимость. Самый высокий процент задолженности у людей со средним уровнем дохода "ниже среднего" 8,65%, самый низкий у людей c уровнем дохода "низкий" -6.90%. Процент задолженности у людей с уровнем дохода "ниже среднего" и "выше среднего" мало отличается, на 0,27%.<br> Значит, уровень дохода  влияет на задолженность при переходе  с уровня дохода "выше среднего" на "высокий". Этот результат может быть использован как фактор при рассмотрении решения о выдаче кредита.

- Как разные цели кредита влияют на его возврат в срок?
<a id="q4"></a>

In [55]:
%%time
#посмотрим еще раз на уникальные значение 'Purpose_cat'. Выведем число клиентов по категории цели кредита
print(data_purpose['purpose_cat'].value_counts())
goals= data_purpose['purpose_cat'].value_counts()


print('Процент клиентов с целью "недвижимость" и задолженностью:{:.2%}'.format(data_purpose[(data_purpose['purpose_cat']=='недвижимость')&(data_purpose['debt']==1)]['purpose_cat'].count()/goals[0]))
print('Процент клиентов с целью "автомобиль" и задолженностью:{:.2%}'.format(data_purpose[(data_purpose['purpose_cat']=='автомобиль')&(data_purpose['debt']==1)]['purpose_cat'].count()/goals[1]))
print('Процент клиентов с целью "образование" и задолженностью:{:.2%}'.format(data_purpose[(data_purpose['purpose_cat']=='образование')&(data_purpose['debt']==1)]['purpose_cat'].count()/goals[2]))
print('Процент клиентов с целью "свадьба" и задолженностью:{:.2%}'.format(data_purpose[(data_purpose['purpose_cat']=='свадьба')&(data_purpose['debt']==1)]['purpose_cat'].count()/goals[3]))
print('Процент клиентов с целью "коммерческий " и задолженностью:{:.2%}'.format(data_purpose[(data_purpose['purpose_cat']=='коммерческий')&(data_purpose['debt']==1)]['purpose_cat'].count()/goals[4]))
print()
#Выведем общий процент клиентов с задолженнотсью, для сравнения
print('Процент клиентов с задолженностью в прошлом в общем:{:.2%}'.format(dolznik/len(data_children['debt'])))



недвижимость    8849
автомобиль      4306
образование     4013
свадьба         2324
коммерческий    1962
Name: purpose_cat, dtype: int64
Процент клиентов с целью "недвижимость" и задолженностью:7.13%
Процент клиентов с целью "автомобиль" и задолженностью:9.36%
Процент клиентов с целью "образование" и задолженностью:9.22%
Процент клиентов с целью "свадьба" и задолженностью:8.00%
Процент клиентов с целью "коммерческий " и задолженностью:7.70%

Процент клиентов с задолженностью в прошлом в общем:8.12%
CPU times: user 27.2 ms, sys: 7.99 ms, total: 35.2 ms
Wall time: 50.9 ms


**Вывод**

Согласно полученным данным, между целью кредита и наличием задолженности в прошлом есть небольшая зависимость. Самый низкий процент задолженности у клиентов с целью "недвижимость" -7.13 %.<br>
Самый высокий процент задолженности у людей с целью "автомобиль" = 9.36%. Это говорит о том что цель кредита может быть использовано как фактор при рассмотрении решения о выдаче кредита.

## Шаг 4. Общий вывод
<a id="summary"></a>

Согласно полученным данным, зависимость между заданными условиями и задолженностью имеется, но в пределах 2% от среднего процента задожденности - 8.12%. Это говорит о том что на факт наличия задолженности в основном влиют другие факторы, помимо рассмотренных<br>
Однако, используя информацию что мы получили и ранжируя эти факторы, мы можем принимать взврешенные решения о выдаче кредитов.<br>
<b>Наш идеальный заемщик -вдовец без детей с низким уровнем дохода, с целью кредита недвижимость<br></b>
<b>Наш нежелательный заемщик - не женат/не замужем, с детьми, уровень дохода ниже среднего, цель кредита автомобиль <br></b>
